In [ ]:
# clone the ADIS repository
!git clone https://github.com/sathishkumar67/ADIS.git
# move the files to the current directory
!mv /kaggle/working/ADIS/* /kaggle/working/
# upgrade pip
!pip install --upgrade pip
# install the required packages
!pip install  -r requirements.txt 
!pip install -U ultralytics

Cloning into 'ADIS'...
remote: Enumerating objects: 1105, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 1105 (delta 70), reused 90 (delta 35), pack-reused 978 (from 1)
Receiving objects: 100% (1105/1105), 39.97 MiB | 37.28 MiB/s, done.
Resolving deltas: 100% (566/566), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.8 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.8/974.8 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 88.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 57.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 122.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 136.4 MB/s eta

In [8]:
# necessary imports
from __future__ import annotations
import os
import joblib
import optuna
from huggingface_hub import hf_hub_download
from utils import unzip_file
from model import YOLO11Model

In [3]:
# Define the global variables
REPO_ID = "pt-sk/ADIS"  # dataset repo ----------------
DATASET_NAME = "balanced_dataset" # dataset name -------------
MODEL_PATH = "yolo11n.pt" # model name ----------------------
FILENAME_IN_REPO = f"{DATASET_NAME}.zip"
LOCAL_DIR = os.getcwd()
TRAIN_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/train"
VAL_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/val"
TEST_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/test"
DATASET_PATH = f"{LOCAL_DIR}/{FILENAME_IN_REPO}"
REPO_TYPE = "dataset"
NUM_CLASSES = 10                                               
CLASSES = ['Cat', 'Cattle', 'Chicken', 'Deer', 'Dog', "Squirrel", 'Eagle', 'Goat', 'Rodents', 'Snake'] 
DATA_YAML_FILE = f"{LOCAL_DIR}/data.yaml"

# download the dataset and unzip it
hf_hub_download(repo_id=REPO_ID, filename=FILENAME_IN_REPO, repo_type=REPO_TYPE, local_dir=LOCAL_DIR)
unzip_file(DATASET_PATH, LOCAL_DIR)

# delete dataset.zip(save memory)
os.remove(DATASET_PATH)

# Get the number of CPU cores
num_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cores}")

balanced_dataset.zip:   0%|          | 0.00/7.04G [00:00<?, ?B/s]

Unzipping: 100%|██████████| 7.07G/7.07G [00:41<00:00, 169MB/s]


Number of CPU cores: 4


In [4]:
# split paths for model
data_yaml = f"""
train: {TRAIN_PATH}
val: {VAL_PATH}
test: {TEST_PATH}

nc: {NUM_CLASSES}
names: {CLASSES}
"""

# write data yaml file
with open(DATA_YAML_FILE, "w") as file:
    file.write(data_yaml)

In [5]:
# Define the objective function
def objective(trial):
    
    # Define callback to report intermediate results
    def on_train_epoch_end(score, epoch):
        trial.report(score, step=epoch)  
        if trial.should_prune():
            raise optuna.TrialPruned()

    callbacks = {
        "on_train_epoch_end" : on_train_epoch_end
    }
    
    # Define hyperparameters using Optuna suggestions
    lr0 = trial.suggest_float("lr0", 1e-5, 1e-2, log=True)
    lrf = trial.suggest_float("lrf", 1e-2, 1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    warmup_momentum = trial.suggest_float("warmup_momentum", 0.0, 0.9)
    momentum = trial.suggest_float("momentum", 0.8, 0.9)
    
    CONFIG_DICT = {
    "task": "detect",
    "mode": "train",
    "bohb": True,
    "custom_callbacks": callbacks,
    "data": DATA_YAML_FILE,
    "batch": 352,  # batch count based on gpu --------------------------
    "imgsz": 320,
    "save": True,
    "device": 0,
    "workers": num_cores,
    "pretrained": True,
    "optimizer": "AdamW",
    "seed": 42,
    "epochs": 20, # epoch count based on the model --------------------------
    "patience": 4,
    "amp": True} # setting amp to false

    # Train YOLO model
    model = YOLO11Model(MODEL_PATH)
    model.train(**CONFIG_DICT, lr0=lr0, lrf=lrf, momentum=momentum, weight_decay=weight_decay, warmup_momentum=warmup_momentum)
    
    # Return validation mAP as the objective value
    return model.score 

In [9]:
NUM_TRIALS = 15

# load the study
study = optuna.create_study(direction='maximize', 
                            sampler=optuna.samplers.TPESampler(), 
                            pruner=optuna.pruners.HyperbandPruner(),
                            study_name="yolo11_tuning",
                            load_if_exists=True)

# Optimize with a callback to stop after NUM_TRIALS complete trials
study.optimize(
    objective,
    n_trials=NUM_TRIALS)

joblib.dump(study, f"{LOCAL_DIR}/optuna_study.pkl")

[I 2025-04-17 08:36:47,052] A new study created in memory with name: yolo11_tuning


Ultralytics 8.3.109 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/working/data.yaml, epochs=20, time=None, patience=4, batch=352, imgsz=320, save=True, save_period=-1, cache=False, device=0, workers=4, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_wid

train: Scanning /kaggle/working/balanced_dataset/train.cache... 18139 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18139/18139 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /kaggle/working/balanced_dataset/val.cache... 2390 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2390/2390 [00:00<?, ?it/s]
[W 2025-04-17 08:36:49,782] Trial 0 failed with parameters: {'lr0': 0.009569778729554767, 'lrf': 0.04831022985454006, 'weight_decay': 0.0002074281770123045, 'warmup_momentum': 0.4583900146814513, 'momentum': 0.8580259384701912} because of the following error: AttributeError("\n            'IterableSimpleNamespace' object has no attribute 'save_hybrid'. This may be caused by a modified or out of date ultralytics\n            'default.yaml' file.\nPlease update your code with 'pip install -U ultralytics' and if necessary replace\n            /usr/local/lib/python3.11/dist-packages/ultralytics/cfg/default.yaml with the latest version from\n            https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/default.yaml\n            ").
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_opti

AttributeError: 
            'IterableSimpleNamespace' object has no attribute 'save_hybrid'. This may be caused by a modified or out of date ultralytics
            'default.yaml' file.
Please update your code with 'pip install -U ultralytics' and if necessary replace
            /usr/local/lib/python3.11/dist-packages/ultralytics/cfg/default.yaml with the latest version from
            https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/default.yaml
            

libpng warning: iCCP: known incorrect sRGB profile
